In [1]:
!pip install -q pymilvus sentence-transformers transformers

/home/octoopt/anaconda3/envs/rag/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [1]:
import pandas as pd


import pymilvus

from transformers import AutoTokenizer, AutoModel
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('keepitreal/vietnamese-sbert')
model = AutoModel.from_pretrained('keepitreal/vietnamese-sbert').to(device)

/home/octoopt/anaconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/octoopt/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
dataset_root = './dataset'
dataset_dir = {
    'corpus': f"{dataset_root}/corpus.csv", 
    'train': f"{dataset_root}/train.csv", 
    'preprocessed': f"{dataset_root}/preprocessed_train.csv", 
    'test': f"{dataset_root}/public_test.csv", 
}

prc_train_df = pd.read_csv(dataset_dir['preprocessed'])
prc_train_df.columns

Index(['question', 'context', 'cid', 'qid'], dtype='object')

In [7]:
prc_train_df.head()

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"Khả năng học tập, nâng cao trình độ\n- Khối lư...",62492,161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,Nội dung lồng ghép vấn đề bình đẳng giới trong...,151154,80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"Điều 20. Ưu đãi về thời gian miễn thuế, giảm t...",75071,124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"Điều kiện được hưởng\nCán bộ quản lý, giáo viê...",225897,146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"Nguyên tắc áp dụng\n1. Trường hợp công chức, v...",68365,6176


# Embedding vector

In [4]:
# Merge question and context into a full sentence of question-answering

prc_train_df['qa_sentence'] = prc_train_df['question'] + " " + prc_train_df['context']
prc_train_df.head()

,question,context,cid,qid,qa_sentence
0,Người học ngành quản lý khai thác công trình t...,"Khả năng học tập, nâng cao trình độ\n- Khối lư...",62492,161615,Người học ngành quản lý khai thác công trình t...
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,Nội dung lồng ghép vấn đề bình đẳng giới trong...,151154,80037,Nội dung lồng ghép vấn đề bình đẳng giới trong...
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"Điều 20. Ưu đãi về thời gian miễn thuế, giảm t...",75071,124074,Sản phẩm phần mềm có được hưởng ưu đãi về thời...
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"Điều kiện được hưởng\nCán bộ quản lý, giáo viê...",225897,146841,Điều kiện để giáo viên trong cơ sở giáo dục mầ...
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"Nguyên tắc áp dụng\n1. Trường hợp công chức, v...",68365,6176,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...


In [31]:
def process_data_in_batches(df, batch_size=10000):
  """Processes data in batches.

  Args:
    df: The Pandas DataFrame to process.
    batch_size: The number of rows to process in each batch.

  Yields:
    A generator that yields batches of the DataFrame.
  """
  for i in range(0, len(df), batch_size):
    yield df[i:i + batch_size]


In [5]:
qa_sentence = prc_train_df['qa_sentence'].tolist()
print(len(qa_sentence))

133568


In [ ]:
from copy import deepcopy

df_copy = deepcopy(prc_train_df)
embeddings = []
idx = 0
for batch in process_data_in_batches(df_copy, batch_size=1000):
    qa_sentence = batch['qa_sentence'].tolist()
    print(len(qa_sentence))
    # Tokenize sentences
    encoded_input = tokenizer(qa_sentence, padding=True, truncation=True, return_tensors='pt').to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).detach().cpu().numpy()

    for sentence, embedding in zip(qa_sentence, sentence_embeddings):
        embeddings.append(embedding)

print(len(embeddings))

# Dataset to Milvus Database

## Preparing data before inserting

In [46]:
vector_df = pd.read_csv(f'./db/vector_db_src.csv', index_col=0)
vector_df.columns

Index(['question', 'context', 'cid', 'qid', 'qa_sentence', 'embeddings', 'id'], dtype='object')

In [48]:
vector_df.embeddings[0]

'[0.111335896, 0.210891947, 0.0511029698, -0.0418972932, 0.0466625504, 0.574945092, -0.0526874512, 0.220133945, 0.151533276, 0.0399256125, -0.00569233857, 0.161261097, 0.272923648, -0.123344555, -0.057279475, 0.0651934743, 0.526156247, 0.206077665, -0.124597728, 0.178007498, -0.0980879068, 0.0854035467, 0.393202245, 0.315725178, -0.191653922, -0.0361217596, 0.105216712, -0.0675185397, -0.173910588, -0.0724113584, 0.2420156, 0.000415267976, -0.121465407, -0.335745275, 0.653289974, -0.111322723, -0.153764561, 0.685039878, 0.296441793, -0.187612265, -0.284840643, -0.316986293, 0.16129975, 0.292015016, -0.120342255, 0.137607634, 0.0747091398, 0.0491073243, -0.000395743147, 0.158131137, -0.495387316, 0.25150454, 0.0596127138, 0.537845373, 0.0918752849, 0.152119726, -0.326908827, 0.0678622201, 0.546638072, 0.174423873, -0.199350834, -0.0181251839, -0.246513382, 0.255967528, 0.262068093, -0.178005025, 0.0898256525, 0.0825907066, 0.573277712, 0.575168431, -0.144680306, 0.0200997461, -0.1054210

In [47]:
type(vector_df.embeddings[0])

str

In [33]:
print(f"Total embedded vectors: {len(vector_df['embeddings'])}")
print(f"Total lengeth of each vector: {len(vector_df['embeddings'][0])}")

Total embedded vectors: 133568
Total lengeth of each vector: 10524


In [10]:
def convert_string_to_float_df(sample): 
    # Remove quotes and square brackets
    string = sample[1:-1]  # Remove the first and last characters

    # Split the string into a list of strings
    float_strings = string.split()

    # Convert each string to a float
    float_list = [float(s) for s in float_strings]

    return float_list

In [11]:
from copy import deepcopy

vector_df_copy = deepcopy(vector_df)

vector_df_copy['embeddings'] = vector_df_copy['embeddings'].apply(lambda x: convert_string_to_float_df(x))
vector_df_copy.columns

Index(['question', 'context', 'cid', 'qid', 'qa_sentence', 'embeddings'], dtype='object')

In [12]:
vector_df_copy['id'] = [i for i in range(len(vector_df_copy))]
vector_df_copy.columns

Index(['question', 'context', 'cid', 'qid', 'qa_sentence', 'embeddings', 'id'], dtype='object')

In [ ]:
# vector_df_copy.to_csv('./dataset/vector_db_src_01.csv')

In [32]:
print(f"Total embedded vectors: {len(vector_df_copy['embeddings'])}")
print(f"Total lengeth of each vector: {len(vector_df_copy['embeddings'][0])}")

Total embedded vectors: 133568
Total lengeth of each vector: 768


## Connect and insert data into Milvus

In [1]:
from pymilvus import __version__

print(__version__)

2.4.9


In [2]:
from pymilvus import MilvusClient

client = MilvusClient("./db/bkai_milvus.db")

Failed to create new connection using: f3a38517193e466caff69acf5f3cd93d


ConnectionConfigException: <ConnectionConfigException: (code=1, message=Open local milvus failed, dir: db is not exists)>

In [62]:
client.drop_collection(collection_name='bkai_vectordb')

In [64]:
client.create_collection(
    collection_name="bkai_vectordb",
    dimension=768,
    primary_field_name="id",
    id_type="int",
    vector_field_name="embeddings",
    auto_id=False,
)

In [65]:
# Divide the whole dataset into batches

idx = 0
for batch in process_data_in_batches(vector_df_copy, batch_size=1000):
    print(idx, len(batch))
    data = [batch.iloc[idx].to_dict() for idx in range(len(batch))]
    # 2. Insert a record
    res = client.insert(
        collection_name="bkai_vectordb",
        data=data
    )
    idx += 1
print('Finish')

0 1000
1 1000
2 1000
3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 1000
10 1000
11 1000
12 1000
13 1000
14 1000
15 1000
16 1000
17 1000
18 1000
19 1000
20 1000
21 1000
22 1000
23 1000
24 1000
25 1000
26 1000
27 1000
28 1000
29 1000
30 1000
31 1000
32 1000
33 1000
34 1000
35 1000
36 1000
37 1000
38 1000
39 1000
40 1000
41 1000
42 1000
43 1000
44 1000
45 1000
46 1000
47 1000
48 1000
49 1000
50 1000
51 1000
52 1000
53 1000
54 1000
55 1000
56 1000
57 1000
58 1000
59 1000
60 1000
61 1000
62 1000
63 1000
64 1000
65 1000
66 1000
67 1000
68 1000
69 1000
70 1000
71 1000
72 1000
73 1000
74 1000
75 1000
76 1000
77 1000
78 1000
79 1000
80 1000
81 1000
82 1000
83 1000
84 1000
85 1000
86 1000
87 1000
88 1000
89 1000
90 1000
91 1000
92 1000
93 1000
94 1000
95 1000
96 1000
97 1000
98 1000
99 1000
100 1000
101 1000
102 1000
103 1000
104 1000
105 1000
106 1000
107 1000
108 1000
109 1000
110 1000
111 1000
112 1000
113 1000
114 1000
115 1000
116 1000
117 1000
118 1000
119 1000
120 1000
121 1000
122 1000
123

# Inference

In [66]:
test_df = pd.read_csv('./db/public_test.csv')
test_df.head()

,question,qid
0,Hiệp hội Công nghiệp ghi âm Việt Nam hoạt động...,98440
1,Báo cáo nghiên cứu khả thi đầu tư xây dựng là ...,105737
2,Lịch khai giảng năm học 2022 - 2023 đối với họ...,106239
3,Số định danh cá nhân có được dùng thay thế các...,79491
4,Trợ cấp đối với Chủ tịch Hội cựu chiến binh cấ...,130557


In [67]:
prompt = test_df.iloc[0]['question']
print(prompt)
encoded_input = tokenizer(prompt, padding=True, truncation=True, return_tensors='pt').to(device)

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask']).detach().cpu().numpy()

Hiệp hội Công nghiệp ghi âm Việt Nam hoạt động trong những lĩnh vực nào?


In [68]:
print(type(sentence_embedding))
print(len(sentence_embedding))
print(sentence_embedding.shape)
print(len(sentence_embedding[0]))

<class 'numpy.ndarray'>
1
(1, 768)
768


In [70]:
search_params = {
    "metric_type": "COSINE",        # Possible values are IP, L2, COSINE, JACCARD, and HAMMING
    "params": {}
}

# Search with limit
res = client.search(
    collection_name="bkai_vectordb",
    data=sentence_embedding,
    limit=10,
    output_fields=['question', 'context', 'cid', 'embeddings'], 
    search_params=search_params
)

In [60]:
res[0][0]

{'id': 38032,
 'distance': 0.6923056840896606,
 'entity': {'question': 'Hiệp hội Công nghiệp ghi âm Việt Nam là tổ chức gì?',
  'context': 'Tôn chỉ, mục đích\\n1. Hiệp hội Công nghiệp ghi âm Việt Nam (sau đây gọi tắt là Hiệp hội) là tổ chức xã hội - nghề nghiệp tự nguyện của tổ chức, công dân Việt Nam đã và đang hoạt động trong lĩnh vực sản xuất bản ghi âm, ghi hình, là chủ sở hữu nắm giữ một phần, một số hoặc toàn bộ các quyền liên quan đối với bản ghi âm, ghi hình (bao gồm các sản phẩm ghi âm, ghi hình, các buổi biểu diễn được định hình) ở Việt Nam theo quy định của pháp luật.\\n2. Hiệp hội hoạt động với mục đích tập hợp, đoàn kết hội viên nhằm hỗ trợ, giúp đỡ lẫn nhau để hoạt động hiệu quả, phát huy đạo đức nghề nghiệp gắn với trách nhiệm xã hội trong việc phát triển sản phẩm bản ghi âm, ghi hình; chống lại các hành vi xâm phạm quyền tác giả, quyền liên quan trong lĩnh vực công nghiệp ghi âm; góp phần thúc đẩy sáng tạo, phổ biến các giá trị âm nhạc, các loại hình nghệ thuật dân tộc 

In [23]:
len(res[0])

10

In [84]:
search_results = []
for r in res[0]: 
    search_result = client.get(
    collection_name="bkai_vectordb",
    ids=r['id']
)
    search_results.append(search_result)

search_results

[data: ["{'id': 38032, 'question': 'Hiệp hội Công nghiệp ghi âm Việt Nam là tổ chức gì?', 'context': 'Tôn chỉ, mục đích\\\\n1. Hiệp hội Công nghiệp ghi âm Việt Nam (sau đây gọi tắt là Hiệp hội) là tổ chức xã hội - nghề nghiệp tự nguyện của tổ chức, công dân Việt Nam đã và đang hoạt động trong lĩnh vực sản xuất bản ghi âm, ghi hình, là chủ sở hữu nắm giữ một phần, một số hoặc toàn bộ các quyền liên quan đối với bản ghi âm, ghi hình (bao gồm các sản phẩm ghi âm, ghi hình, các buổi biểu diễn được định hình) ở Việt Nam theo quy định của pháp luật.\\\\n2. Hiệp hội hoạt động với mục đích tập hợp, đoàn kết hội viên nhằm hỗ trợ, giúp đỡ lẫn nhau để hoạt động hiệu quả, phát huy đạo đức nghề nghiệp gắn với trách nhiệm xã hội trong việc phát triển sản phẩm bản ghi âm, ghi hình; chống lại các hành vi xâm phạm quyền tác giả, quyền liên quan trong lĩnh vực công nghiệp ghi âm; góp phần thúc đẩy sáng tạo, phổ biến các giá trị âm nhạc, các loại hình nghệ thuật dân tộc và tinh hoa âm nhạc thế giới tới c